## Final Project: Phase 3 - Clustering Pipeline Optimization
Spring 2024  
Group: Michael Massone and Joseph Nelson Farrell   
DS 5230 Unsupervised Machine Learning  
Professor Steven Morin, PhD  
Due: 04/21/2024  
___

In [1]:
## Libraries

# base
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random


# preprocessing
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

# data
from sklearn.datasets import load_digits

# clustering
from sklearn.cluster import DBSCAN, KMeans

#external indices
from sklearn.metrics.cluster import adjusted_rand_score, contingency_matrix

# runtime and run tracking
import time
from datetime import datetime

# pathing
from pathlib import Path
import os
import sys

import warnings

# Disable runtime warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)



___

### Start Timer

In [2]:
# track runtime
start = time.time()

___

### Define File Paths

In [3]:
# define path
nb_path = Path(os.getcwd())
print(nb_path)
path = str(nb_path.parent)
print(path)

# path to figs folder
figs_path = path + '/figs'

# path to data
data_path= path + '/data'

# path to src folder
src_path = path + '/src'
print(src_path)

# sys path
sys.path.append(src_path)

/Users/mikey/LocalFiles/DS5230/final_project/DS5230-final/notebooks
/Users/mikey/LocalFiles/DS5230/final_project/DS5230-final
/Users/mikey/LocalFiles/DS5230/final_project/DS5230-final/src


____

### Import Modules

In [4]:
# modules and util
import dimensionality_reduction as dr
import clustering as cl
from cluster_utils import *
from external_indices_utils import *

___

### Parameters

In [5]:
# transformed data csv file name
design_file = '/curated/shuffled_target.csv'

# target vector csv file name
target_file = "/curated/beans_target.csv"

# sampled target dataframe csv loc
target_vector_shuffled_filename = f'/sampled/sampled_target{str(datetime.now()).replace(" ", "")}.csv'

# sampled design dataframe csv loc
design_matrix_shuffled_filename = f'/sampled/sampled_design{str(datetime.now()).replace(" ", "")}.csv'

# all results dataframe csv loc
all_results_file = f'/results/all_results_{str(datetime.now()).replace(" ", "")}.csv'

# filtered results dataframe csv loc
filtered_results_file = f'/results/filtered_results_{str(datetime.now()).replace(" ", "")}.csv'


____

### Load Data

In [6]:
design_file = '/curated/shuffled_target.csv'
target_vector_shuffled = pd.read_csv(data_path + design_file)

target_file = '/curated/shuffled_design.csv'
design_matrix_shuffled = pd.read_csv(data_path + target_file)


In [7]:
#design_matrix_shuffled = design_matrix_shuffled.loc[:, :'numerical__ShapeFactor1']

____

### Sample Data

In [8]:
sample = False
if sample:
    # num samples
    n = 1000

    # random seed
    rand_seed = 42

    # sample target and save copy to csv
    target_vector_shuffled = target_vector_shuffled.sample(n=n, random_state=rand_seed)
    target_vector_shuffled.to_csv(data_path + target_vector_shuffled_filename, index=False)
    print("Sampled target dataframe saved at:", data_path + target_vector_shuffled_filename)


     # sample target and save copy to csv
    design_matrix_shuffled = design_matrix_shuffled.sample(n=n, random_state=rand_seed)
    design_matrix_shuffled.to_csv(data_path + design_matrix_shuffled_filename, index=False)
    print("Sampled design dataframe saved at:", data_path + design_matrix_shuffled_filename)


___

### Create capX

In [9]:
# drop ID col
design_matrix_shuffled_noID = design_matrix_shuffled.drop('ID', axis=1)

# convert to ndarray
cap_x = design_matrix_shuffled_noID.to_numpy()
cap_x = design_matrix_shuffled
print(f'cap_x shape: {cap_x.shape}')
cap_x.shape

cap_x shape: (13611, 17)


(13611, 17)

___

### Select Hyperparameters for Gridsearch

In [10]:
# select hyperparams
params1=False
params2=False
params3=False
params4=True

if params1:

    min_dist_list = [0.0, 0.01, 0.1, 0.25, 0.5]
    n_neighbors_list = [10, 25, 50, 100, 200]
    metric_list =['euclidean', 'canberra', 'mahalanobis', 'correlation']
    n_components_list = range(2, 4)


if params2:

    min_dist_list = [0.5]
    n_neighbors_list = [100]
    metric_list = ['euclidean']
    n_components_list = [3]


if params3:

    min_dist_list = [0.0, 0.1, 0.5]
    n_neighbors_list = [10, 25, 100, 200]
    metric_list =['euclidean', 'canberra', 'chebyshev', 'correlation']
    n_components_list = range(2, 4)


if params4:

    min_dist_list = [0.0]
    n_neighbors_list = [50, 100, 250, 500, 1000]
    metric_list =['cosine', 'euclidean']
    n_components_list = range(2, 9)




___

### Gridsearch Algorithm

In [11]:
runs = len(n_components_list)*len(min_dist_list)*len(n_neighbors_list)*len(metric_list)
run = 0
df_row_dict_list = []
for n_components in n_components_list:
  for min_dist in min_dist_list:
    for n_neighbors in n_neighbors_list: 
      for metric in metric_list:
        run +=1
        print('*'*100)
        print(f'Run {run} of {runs}')
        results_dict = dr.umap_dim_red(cap_x, n_neighbors, min_dist, metric, n_components) 
        df_row_dict = cl.clustering(results_dict)
        df_row_dict_list.append(df_row_dict)
results_df = pd.DataFrame(df_row_dict_list)


****************************************************************************************************
Run 1 of 70


/Users/mikey/miniconda3/envs/usml_base_ds2/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


****************************************************************************************************
****************************************************************************************************
Hyperparameters:
n_neighbors:  50
min_dist:  0.0
metric:  cosine
n_components:  2
Hopkin's Statistic = 0.08096146089983365
Silhouette Score:  0.7944241
Fail: Kmeans did not successfully cluster.
DBSCAN
Number of Clusters:  2
Validity Index:  0.9074445569880059
[0 1]
Noise Ratio:  0.0
****************************************************************************************************
Run 2 of 70


/Users/mikey/miniconda3/envs/usml_base_ds2/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


****************************************************************************************************
****************************************************************************************************
Hyperparameters:
n_neighbors:  50
min_dist:  0.0
metric:  euclidean
n_components:  2
Hopkin's Statistic = 0.08305082503668063
Silhouette Score:  0.43277013
Fail: Kmeans did not successfully cluster.
DBSCAN
Number of Clusters:  5
Validity Index:  -0.313180872122108
[0 1 2 3 4]
Noise Ratio:  0.0
****************************************************************************************************
Run 3 of 70


...

****************************************************************************************************
****************************************************************************************************
Hyperparameters:
n_neighbors:  500
min_dist:  0.0
metric:  euclidean
n_components:  8
Hopkin's Statistic = 0.0026333681044570178
Silhouette Score:  0.51396453
Fail: Kmeans did not successfully cluster.
DBSCAN
Number of Clusters:  3
Validity Index:  0.5379647925988382
[0 1 2]
Noise Ratio:  0.0
****************************************************************************************************
Run 69 of 70


/Users/mikey/miniconda3/envs/usml_base_ds2/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


****************************************************************************************************
****************************************************************************************************
Hyperparameters:
n_neighbors:  1000
min_dist:  0.0
metric:  cosine
n_components:  8
Hopkin's Statistic = 0.009970730076281634
Silhouette Score:  0.92963475
Fail: Kmeans did not successfully cluster.
DBSCAN
Number of Clusters:  4
Validity Index:  0.9997873503351963
[0 1 2 3]
Noise Ratio:  0.0
****************************************************************************************************
Run 70 of 70


/Users/mikey/miniconda3/envs/usml_base_ds2/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


****************************************************************************************************
****************************************************************************************************
Hyperparameters:
n_neighbors:  1000
min_dist:  0.0
metric:  euclidean
n_components:  8
Hopkin's Statistic = 0.0025787904762104696
Silhouette Score:  0.5150345
Test2 Pass: Kmeans successfully clustered.
Number of Clusters:  9


___

## All Results

In [12]:
results_df.to_csv(data_path + all_results_file, index=False)
print("Filtered results dataframe saved at:", data_path + all_results_file)

# display max rows
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
results_df

Filtered results dataframe saved at: /Users/mikey/LocalFiles/DS5230/final_project/DS5230-final/data/results/all_results_2024-04-1722:42:11.819198.csv


,algo,n_clusters_found,n_clusters_db_score_is_min,n_clusters_ch_score_is_max,n_clusters_silhouette_score_is_max,silhouette_score,hopkins_statistic,umap_n_neighbors,umap_min_dist,umap_metric,umap_n_components,trustworthiness,eps,dbscan_min_samples,dbscan_metric,validity_index,noise_ratio,cluster_labels
0,dbscan,2,NaN,NaN,NaN,NaN,0.080961,50,0.0,cosine,2,0.893050,0.100099,5.0,cosine,0.907445,0.000000,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1,dbscan,5,NaN,NaN,NaN,NaN,0.083051,50,0.0,euclidean,2,0.999435,0.171741,5.0,euclidean,-0.313181,0.000000,"[0, 0, 0, 0, 1, 0, 2, 0, 0, 1, 1, 1, 1, 3, 1, ..."
2,dbscan,12,NaN,NaN,NaN,NaN,0.071899,100,0.0,cosine,2,0.895107,0.119371,5.0,euclidean,0.584668,0.006906,"[0, 6, 0, 1, 2, 0, 3, 0, 4, 2, 2, 2, 2, 5, 2, ..."
3,dbscan,5,NaN,NaN,NaN,NaN,0.070136,100,0.0,euclidean,2,0.999229,0.145034,5.0,euclidean,-0.317923,0.000000,"[0, 0, 0, 0, 1, 0, 2, 0, 0, 1, 1, 3, 1, 3, 1, ..."
4,dbscan,2,NaN,NaN,NaN,NaN,0.058392,250,0.0,cosine,2,0.897391,0.101670,5.0,cosine,0.947845,0.000000,"[0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, ..."
5,dbscan,70,NaN,NaN,NaN,NaN,0.061616,250,0.0,euclidean,2,0.998591,0.074673,5.0,euclidean,-0.442206,0.001690,"[0, 1, 2, 3, 4, 5, 6, 0, 7, 4, 8, 9, 10, 11, 4..."
6,dbscan,3,NaN,NaN,NaN,NaN,0.058658,500,0.0,cosine,2,0.875679,0.135633,5.0,cosine,0.991350,0.000000,"[0, 1, 0, 0, 1, 0, 1, 0, 2, 1, 1, 1, 1, 1, 1, ..."
7,dbscan,119,NaN,NaN,NaN,NaN,0.056533,500,0.0,euclidean,2,0.996734,0.066758,5.0,euclidean,-0.227907,0.001763,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
8,dbscan,4,NaN,NaN,NaN,NaN,0.063390,1000,0.0,cosine,2,0.865591,0.127078,5.0,cosine,0.999653,0.000000,"[0, 1, 0, 2, 1, 0, 1, 0, 3, 1, 1, 1, 1, 1, 1, ..."
9,dbscan,33,NaN,NaN,NaN,NaN,0.053227,1000,0.0,euclidean,2,0.990320,0.073935,5.0,euclidean,-0.512701,0.000220,"[0, 1, 2, 3, 4, 2, 5, 0, 6, 7, 4, 8, 9, 10, 11..."


In [13]:
# set max rows back to default
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

____

### Get n Best Results (kmeans and DBSCAN)

#### Kmeans Results

In [25]:
n=1

kmeans_results_df = results_df[results_df['algo'] == 'k_means']
kmeans_results_df = kmeans_results_df.sort_values(by='silhouette_score', ascending=False)
best_kmeans_results_df = kmeans_results_df.head(n)
best_kmeans_results_df

,algo,n_clusters_found,n_clusters_db_score_is_min,n_clusters_ch_score_is_max,n_clusters_silhouette_score_is_max,silhouette_score,hopkins_statistic,umap_n_neighbors,umap_min_dist,umap_metric,umap_n_components,trustworthiness,eps,dbscan_min_samples,dbscan_metric,validity_index,noise_ratio,cluster_labels,combined_score
59,k_means,8,15.0,15.0,15.0,0.516789,0.002807,1000,0.0,euclidean,7,0.999931,NaN,NaN,NaN,NaN,NaN,"[12, 7, 5, 5, 1, 5, 10, 12, 4, 8, 14, 11, 6, 0...",0.516789


#### DBSCAN Results

In [26]:
dbscan_results_df = results_df[results_df['algo'] == 'dbscan']
dbscan_results_df = dbscan_results_df.sort_values(by='validity_index', ascending=False)
best_dbscan_results_df = dbscan_results_df.head(n)
best_dbscan_results_df

,algo,n_clusters_found,n_clusters_db_score_is_min,n_clusters_ch_score_is_max,n_clusters_silhouette_score_is_max,silhouette_score,hopkins_statistic,umap_n_neighbors,umap_min_dist,umap_metric,umap_n_components,trustworthiness,eps,dbscan_min_samples,dbscan_metric,validity_index,noise_ratio,cluster_labels,combined_score
58,dbscan,4,NaN,NaN,NaN,NaN,0.00941,1000,0.0,cosine,7,0.84725,0.265387,5.0,cosine,0.999869,0.0,"[0, 1, 0, 2, 1, 0, 1, 0, 3, 1, 1, 1, 1, 1, 1, ...",0.999869


In [27]:
filtered_results_df = pd.concat([best_dbscan_results_df, best_kmeans_results_df])
filtered_results_df

,algo,n_clusters_found,n_clusters_db_score_is_min,n_clusters_ch_score_is_max,n_clusters_silhouette_score_is_max,silhouette_score,hopkins_statistic,umap_n_neighbors,umap_min_dist,umap_metric,umap_n_components,trustworthiness,eps,dbscan_min_samples,dbscan_metric,validity_index,noise_ratio,cluster_labels,combined_score
58,dbscan,4,NaN,NaN,NaN,NaN,0.009410,1000,0.0,cosine,7,0.847250,0.265387,5.0,cosine,0.999869,0.0,"[0, 1, 0, 2, 1, 0, 1, 0, 3, 1, 1, 1, 1, 1, 1, ...",0.999869
59,k_means,8,15.0,15.0,15.0,0.516789,0.002807,1000,0.0,euclidean,7,0.999931,NaN,NaN,NaN,NaN,NaN,"[12, 7, 5, 5, 1, 5, 10, 12, 4, 8, 14, 11, 6, 0...",0.516789


____

### Save Filtered Results to CSV

In [28]:
filtered_results_df.to_csv(data_path + filtered_results_file, index=False)
print("Filtered results dataframe saved at:", data_path + filtered_results_file)

Filtered results dataframe saved at: /Users/mikey/LocalFiles/DS5230/final_project/DS5230-final/data/results/filtered_results_2024-04-1722:42:11.819218.csv


___

## Runtime

In [17]:
finish = time.time()
hours = int((finish - start) // 3600)
minutes = int(((finish - start) % 3600) // 60)
seconds = int((finish - start) % 60)
print(f"Total Run Time(hh:mm.ss): {hours:02d}:{minutes:02d}.{seconds:02d}")

Total Run Time(hh:mm.ss): 07:59.23
